# Data Cleansing

## Import necessary libraries

In [1]:
%pip install -qq -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Add current directory to Python path for imports
import os
import sys

# Add the parent directory (project root) to Python path so we can import from src
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.append(project_root)

In [3]:
import numpy as np
import pandas as pd
import yaml

## Loading Datasets

In [4]:
# Load data using configuration file
configs_path = os.path.join("..", "configs", "configs.yaml")

with open(configs_path, encoding="utf-8") as config_file:
    config = yaml.safe_load(config_file)
    data_filename = config["data"]["raw_data_path"]

# Make path relative to notebook location
data_path = os.path.join("..", data_filename)

df = pd.read_csv(data_path)
df.head()

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-10-14 10:45:27.713884+00,เสร็จสิ้น,NaN,0,2022-09-08 08:35:43.784519+00
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00


---

## Testing Transformers

### Date Transformer

In [5]:
from src.pipelines import DateTransformer

dt = DateTransformer()
df_transformed = pd.DataFrame(dt.fit_transform(df))

df_transformed.head(10)

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,state,star,count_reopen,timestamp_date,timestamp_month,timestamp_year,last_activity_date,last_activity_month,last_activity_year
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,เสร็จสิ้น,NaN,0,3,9,2021,4,6,2022
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.66709,13.67891",189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,เสร็จสิ้น,4.0,0,19,9,2021,21,6,2022
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,"100.52649,13.72060",191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,เสร็จสิ้น,NaN,0,26,9,2021,6,6,2022
3,2021-9U2NJT,{น้ำท่วม},"เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ",น้ำท่วม,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53099,13.81853",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,เสร็จสิ้น,NaN,0,14,10,2021,8,9,2022
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,เสร็จสิ้น,5.0,0,9,12,2021,12,8,2022
5,2021-4D9Y98,{},"เขตลาดพร้าว,การไฟฟ้านครหลวง เขตนวลจันทร์",หน้าปากซอย ลาดพร้าววังหิน26,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.59131,13.80910",17/73 17/73 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขต...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,เสร็จสิ้น,NaN,0,13,12,2021,14,3,2023
6,2021-7U9RED,{},เขตดุสิต,ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.50848,13.77832",627 ถนนสามเสน แขวง ดุสิต เขตดุสิต กรุงเทพมหานค...,ดุสิต,ดุสิต,กรุงเทพมหานคร,เสร็จสิ้น,5.0,0,17,12,2021,17,5,2023
7,2021-8N9ZP8,{ความสะอาด},"เขตประเวศ,ฝ่ายเทศกิจ เขตประเวศ,ฝ่ายรักษาความสะ...",คนเอาขยะมาทิ้งจนกลายเป็นกองขยะค่ะ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.64690,13.67083",110 ซอย มีสุข แขวง หนองบอน เขต ประเวศ กรุงเทพม...,หนองบอน,ประเวศ,กรุงเทพมหานคร,เสร็จสิ้น,2.0,0,18,12,2021,26,11,2024
8,2021-7K6QA3,{},"เขตประเวศ,ฝ่ายเทศกิจ เขตประเวศ",ระยะหลังๆ นี้ พบเจอรถสิบล้อขนดิน ใส่ยางบังโคลน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.65617,13.72812",208/22 ถ. พัฒนาการ แขวง ประเวศ เขต ประเวศ กรุง...,ประเวศ,ประเวศ,กรุงเทพมหานคร,เสร็จสิ้น,2.0,0,21,12,2021,24,6,2022
9,2021-8BTWZB,{ท่อระบายน้ำ},"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",ขอแจ้งเรื่องท่อระบายน้ำบนถนนในซอยเสียหาย เป็นร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.65440,13.68158",70 ซอย เฉลิมพระเกียรติ ร. 9 แขวง หนองบอน เขต ป...,หนองบอน,ประเวศ,กรุงเทพมหานคร,เสร็จสิ้น,5.0,0,22,12,2021,20,6,2022


### Province Transformer

In [6]:
from src.pipelines import ProvinceTransformer

pt = ProvinceTransformer()
df_transformed = pd.DataFrame(pt.fit_transform(df))
filtered_values = pt.get_filtered_values()

print(f"Filtered values (not found in whitelist): {filtered_values}")

df_province_count = pd.DataFrame(df_transformed.province.value_counts())
df_province_count.head(10)

Filtered values (not found in whitelist): ['', 'nan', 'Sahel', 'Perak', 'Riau', 'Lac', 'Borno', 'Steiermark', 'Tillabéri']


,count
province,
กรุงเทพมหานคร,785662
นนทบุรี,261
สมุทรปราการ,244
ปทุมธานี,110
สมุทรสาคร,45
นครปฐม,30
นครราชสีมา,21
ภูเก็ต,14
เพชรบุรี,13


### District and Subdistrict Transformer

In [7]:
from src.pipelines import DistrictSubdistrictTransformer

dst = DistrictSubdistrictTransformer()
df_transformed = pd.DataFrame(dst.fit_transform(df))

df_area_count = pd.DataFrame(df_transformed[["district", "subdistrict"]].value_counts())
df_area_count.head(10)

,,count
district,subdistrict,
สวนหลวง,สวนหลวง,19663
ดินแดง,ดินแดง,19477
พญาไท,สามเสนใน,14205
บางกะปิ,หัวหมาก,14198
คลองเตย,คลองเตย,13565
ประเวศ,ประเวศ,13020
บางขุนเทียน,แสมดำ,12759
จตุจักร,จอมพล,12719
บางเขน,อนุสาวรีย์,12659


### Coordinate Transformer

In [8]:
from src.pipelines import CoordinateTransformer

ct = CoordinateTransformer()
df_transformed = pd.DataFrame(ct.fit_transform(df))

df_transformed.head(10)

,ticket_id,type,organization,comment,photo,photo_after,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,longitude,latitude
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00,100.66709,13.67891
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00,100.52649,13.72060
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00,100.59165,13.82280
5,2021-4D9Y98,{},"เขตลาดพร้าว,การไฟฟ้านครหลวง เขตนวลจันทร์",หน้าปากซอย ลาดพร้าววังหิน26,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,17/73 17/73 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขต...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00,เสร็จสิ้น,NaN,0,2023-03-14 12:09:14.947437+00,100.59131,13.80910
6,2021-7U9RED,{},เขตดุสิต,ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,627 ถนนสามเสน แขวง ดุสิต เขตดุสิต กรุงเทพมหานค...,ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00,เสร็จสิ้น,5.0,0,2023-05-17 06:11:32.463984+00,100.50848,13.77832
7,2021-8N9ZP8,{ความสะอาด},"เขตประเวศ,ฝ่ายเทศกิจ เขตประเวศ,ฝ่ายรักษาความสะ...",คนเอาขยะมาทิ้งจนกลายเป็นกองขยะค่ะ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,110 ซอย มีสุข แขวง หนองบอน เขต ประเวศ กรุงเทพม...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-18 14:50:52.437512+00,เสร็จสิ้น,2.0,0,2024-11-26 04:17:39.760344+00,100.64690,13.67083
8,2021-7K6QA3,{},"เขตประเวศ,ฝ่ายเทศกิจ เขตประเวศ",ระยะหลังๆ นี้ พบเจอรถสิบล้อขนดิน ใส่ยางบังโคลน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,208/22 ถ. พัฒนาการ แขวง ประเวศ เขต ประเวศ กรุง...,ประเวศ,ประเวศ,กรุงเทพมหานคร,2021-12-21 23:03:58.450912+00,เสร็จสิ้น,2.0,0,2022-06-24 06:32:34.671236+00,100.65617,13.72812
9,2021-8BTWZB,{ท่อระบายน้ำ},"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",ขอแจ้งเรื่องท่อระบายน้ำบนถนนในซอยเสียหาย เป็นร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,70 ซอย เฉลิมพระเกียรติ ร. 9 แขวง หนองบอน เขต ป...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-22 10:15:33.294829+00,เสร็จสิ้น,5.0,0,2022-06-20 13:12:04.99444+00,100.65440,13.68158
10,2021-AKJBCU,{},"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",แจ้งเรื่องพื้นผิวถนนเสียหาย มีการพยายามซ่อมแซม...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,55 ถนน ศรีนครินทร์ แขวง หนองบอน เขต ประเวศ กรุ...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-23 10:26:48.86825+00,เสร็จสิ้น,NaN,0,2022-09-28 08:35:36.703421+00,100.64844,13.68735
11,2021-EQWYWT,{ความสะอาด},"เขตประเวศ,สำนักสิ่งแวดล้อม กทม.,ฝ่ายสิ่งแวดล้อ...",ปัญหากลิ่นขยะจากโรงขยะ 800 ตัน อ่อนนุช 86 ครับ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,22/1 ซอย อ่อนนุช 86 แยก 2 แขวง ประเวศ เขต ประเ...,ประเวศ,ประเวศ,กรุงเทพมหานคร,2021-12-28 03:59:06.003252+00,เสร็จสิ้น,4.0,0,2022-07-12 06:50:02.31526+00,100.68837,13.71887


### Address Transformer

In [9]:
from src.pipelines import AddressTransformer

at = AddressTransformer()
df_transformed = pd.DataFrame(at.fit_transform(df))

df_transformed.head(10)

,ticket_id,type,organization,comment,photo,photo_after,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,longitude,latitude
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 14:56:08.924992+00,เสร็จสิ้น,4.0,0,2022-06-21 08:21:09.532782+00,100.66709,13.67891
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 05:03:52.594898+00,เสร็จสิ้น,NaN,0,2022-06-06 01:17:12.272904+00,100.52649,13.72060
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00,100.59165,13.82280
5,2021-4D9Y98,{},"เขตลาดพร้าว,การไฟฟ้านครหลวง เขตนวลจันทร์",หน้าปากซอย ลาดพร้าววังหิน26,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,17/73 17/73 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขต...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00,เสร็จสิ้น,NaN,0,2023-03-14 12:09:14.947437+00,100.59131,13.80910
6,2021-7U9RED,{},เขตดุสิต,ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,627 ถนนสามเสน แขวง ดุสิต เขตดุสิต กรุงเทพมหานค...,ดุสิต,ดุสิต,กรุงเทพมหานคร,2021-12-17 08:46:02.610983+00,เสร็จสิ้น,5.0,0,2023-05-17 06:11:32.463984+00,100.50848,13.77832
7,2021-8N9ZP8,{ความสะอาด},"เขตประเวศ,ฝ่ายเทศกิจ เขตประเวศ,ฝ่ายรักษาความสะ...",คนเอาขยะมาทิ้งจนกลายเป็นกองขยะค่ะ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,110 ซอย มีสุข แขวง หนองบอน เขต ประเวศ กรุงเทพม...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-18 14:50:52.437512+00,เสร็จสิ้น,2.0,0,2024-11-26 04:17:39.760344+00,100.64690,13.67083
8,2021-7K6QA3,{},"เขตประเวศ,ฝ่ายเทศกิจ เขตประเวศ",ระยะหลังๆ นี้ พบเจอรถสิบล้อขนดิน ใส่ยางบังโคลน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,208/22 ถ. พัฒนาการ แขวง ประเวศ เขต ประเวศ กรุง...,ประเวศ,ประเวศ,กรุงเทพมหานคร,2021-12-21 23:03:58.450912+00,เสร็จสิ้น,2.0,0,2022-06-24 06:32:34.671236+00,100.65617,13.72812
9,2021-8BTWZB,{ท่อระบายน้ำ},"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",ขอแจ้งเรื่องท่อระบายน้ำบนถนนในซอยเสียหาย เป็นร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,70 ซอย เฉลิมพระเกียรติ ร. 9 แขวง หนองบอน เขต ป...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-22 10:15:33.294829+00,เสร็จสิ้น,5.0,0,2022-06-20 13:12:04.99444+00,100.65440,13.68158
10,2021-AKJBCU,{},"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",แจ้งเรื่องพื้นผิวถนนเสียหาย มีการพยายามซ่อมแซม...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,55 ถนน ศรีนครินทร์ แขวง หนองบอน เขต ประเวศ กรุ...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-23 10:26:48.86825+00,เสร็จสิ้น,NaN,0,2022-09-28 08:35:36.703421+00,100.64844,13.68735
11,2021-EQWYWT,{ความสะอาด},"เขตประเวศ,สำนักสิ่งแวดล้อม กทม.,ฝ่ายสิ่งแวดล้อ...",ปัญหากลิ่นขยะจากโรงขยะ 800 ตัน อ่อนนุช 86 ครับ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,22/1 ซอย อ่อนนุช 86 แยก 2 แขวง ประเวศ เขต ประเ...,ประเวศ,ประเวศ,กรุงเทพมหานคร,2021-12-28 03:59:06.003252+00,เสร็จสิ้น,4.0,0,2022-07-12 06:50:02.31526+00,100.68837,13.71887


### State to Status Transformer

In [10]:
from src.pipelines import StateToStatusTransformer

stst = StateToStatusTransformer()
df_transformed = pd.DataFrame(stst.fit_transform(df_transformed))

df_status_count = pd.DataFrame(df_transformed.status.value_counts())
df_status_count.head(10)

,count
status,
done,582505
in-progress,115811
pending,9961


---

## Applying Cleansing Pipeline

In [11]:
from src.pipelines import CleansingPipeline

cleansing_pipeline = CleansingPipeline()

df_cleansed = pd.DataFrame(cleansing_pipeline.fit_transform(df))
df_cleansed.head(10)

,ticket_id,type,organization,comment,photo,photo_after,address,subdistrict,district,province,...,star,count_reopen,timestamp_date,timestamp_month,timestamp_year,last_activity_date,last_activity_month,last_activity_year,longitude,latitude
1,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}","เขตประเวศ,ฝ่ายโยธา เขตประเวศ",น้ำท่วมเวลาฝนตกและทะลุเข้าบ้านเดือดร้อนมากทุกๆ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,189 เฉลิมพระเกียรติ ร.9 แขวง หนองบอน เขต ประเว...,หนองบอน,ประเวศ,กรุงเทพมหานคร,...,4.0,0,19,9,2021,21,6,2022,100.66709,13.67891
2,2021-7XATFA,{สะพาน},เขตสาทร,สะพานลอยปรับปรุงไม่เสร็จตามกำหนด\nปากซอย สาทร12,https://storage.googleapis.com/traffy_public_b...,NaN,191/1 ถนน สาทรเหนือ แขวง สีลม เขตบางรัก กรุงเท...,ยานนาวา,สาทร,กรุงเทพมหานคร,...,NaN,0,26,9,2021,6,6,2022,100.52649,13.72060
4,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,...,5.0,0,9,12,2021,12,8,2022,100.59165,13.82280
5,2021-4D9Y98,{},"เขตลาดพร้าว,การไฟฟ้านครหลวง เขตนวลจันทร์",หน้าปากซอย ลาดพร้าววังหิน26,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,17/73 17/73 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขต...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,...,NaN,0,13,12,2021,14,3,2023,100.59131,13.80910
6,2021-7U9RED,{},เขตดุสิต,ยังไม่มีหน่วยงานไหนมาดูแลครับ รถจะเชี่ยวหลายคน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,627 ถนนสามเสน แขวง ดุสิต เขตดุสิต กรุงเทพมหานค...,ดุสิต,ดุสิต,กรุงเทพมหานคร,...,5.0,0,17,12,2021,17,5,2023,100.50848,13.77832
7,2021-8N9ZP8,{ความสะอาด},"เขตประเวศ,ฝ่ายเทศกิจ เขตประเวศ,ฝ่ายรักษาความสะ...",คนเอาขยะมาทิ้งจนกลายเป็นกองขยะค่ะ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,110 ซอย มีสุข แขวง หนองบอน เขต ประเวศ กรุงเทพม...,หนองบอน,ประเวศ,กรุงเทพมหานคร,...,2.0,0,18,12,2021,26,11,2024,100.64690,13.67083
8,2021-7K6QA3,{},"เขตประเวศ,ฝ่ายเทศกิจ เขตประเวศ",ระยะหลังๆ นี้ พบเจอรถสิบล้อขนดิน ใส่ยางบังโคลน...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,208/22 ถ. พัฒนาการ แขวง ประเวศ เขต ประเวศ กรุง...,ประเวศ,ประเวศ,กรุงเทพมหานคร,...,2.0,0,21,12,2021,24,6,2022,100.65617,13.72812
9,2021-8BTWZB,{ท่อระบายน้ำ},"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",ขอแจ้งเรื่องท่อระบายน้ำบนถนนในซอยเสียหาย เป็นร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,70 ซอย เฉลิมพระเกียรติ ร. 9 แขวง หนองบอน เขต ป...,หนองบอน,ประเวศ,กรุงเทพมหานคร,...,5.0,0,22,12,2021,20,6,2022,100.65440,13.68158
10,2021-AKJBCU,{},"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",แจ้งเรื่องพื้นผิวถนนเสียหาย มีการพยายามซ่อมแซม...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,55 ถนน ศรีนครินทร์ แขวง หนองบอน เขต ประเวศ กรุ...,หนองบอน,ประเวศ,กรุงเทพมหานคร,...,NaN,0,23,12,2021,28,9,2022,100.64844,13.68735
11,2021-EQWYWT,{ความสะอาด},"เขตประเวศ,สำนักสิ่งแวดล้อม กทม.,ฝ่ายสิ่งแวดล้อ...",ปัญหากลิ่นขยะจากโรงขยะ 800 ตัน อ่อนนุช 86 ครับ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,22/1 ซอย อ่อนนุช 86 แยก 2 แขวง ประเวศ เขต ประเ...,ประเวศ,ประเวศ,กรุงเทพมหานคร,...,4.0,0,28,12,2021,12,7,2022,100.68837,13.71887


In [12]:
df_cleansed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 708277 entries, 1 to 787024
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ticket_id            700050 non-null  object 
 1   type                 708193 non-null  object 
 2   organization         708174 non-null  object 
 3   comment              700050 non-null  object 
 4   photo                708185 non-null  object 
 5   photo_after          578977 non-null  object 
 6   address              700050 non-null  object 
 7   subdistrict          708277 non-null  object 
 8   district             708277 non-null  object 
 9   province             708276 non-null  object 
 10  status               708277 non-null  object 
 11  star                 247328 non-null  float64
 12  count_reopen         708277 non-null  int64  
 13  timestamp_date       708276 non-null  Int64  
 14  timestamp_month      708276 non-null  Int64  
 15  timestamp_year       7

In [13]:
pd.DataFrame.to_csv(
    df_cleansed,
    os.path.join("..", "data", "processed", "cleansed_data.csv"),
    index=False,
)

---